In [ ]:
import math
import numpy as np

import pandas as pd
import scipy.io as io
import os 
import csv

from tqdm import tqdm 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.tensorboard import SummaryWriter

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from FCAEClass import FCAE
from RespDataClass import RespData

### 辅助函数
- 预测网络结果
- 制造白噪声
- 计算误差，MSE
- 绘制误差分布

In [ ]:
def predict(test_loader, model, device):
    model.eval()
    preds = torch.Tensor([])
    for x in tqdm(test_loader):
        x = x.float().to(device)
        with torch.no_grad():
            pred = model(x)
            preds = torch.concat((preds, pred.detach().to('cpu')))
    # prds = torch.cat(preds, dim=0).numpy()
    return preds


In [ ]:
from utils import getMSE, draw_hitogram

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mouseID = 1

Respdata = io.loadmat('./Main experiments/Mouse 1/RespData.mat')
Respdata = Respdata['RespData'][0][0][0][0][0][0]
repeatNum, frameNum, cellNum, sessionNum = Respdata.shape

stride = 6


#### 加载数据

In [ ]:
PDG_Data = [np.load(f'./myData/Day{i*7}_PDG/PDG_mouse{mouseID}_FCAE.npy') for i in range(sessionNum)]
MOV_Data = np.load(f'./myData/Day0_MOV/MOV_mouse1_FCAE.npy')
# whiteNoisy_Data = np.random.normal(0, 1, (10000, stride*cellNum))

In [ ]:
PDG_Dataset = [RespData(d) for d in PDG_Data]
MOV_Dataset = RespData(MOV_Data)
# whiteNoisy_Dataset = RespData(whiteNoisy_Data)

PDG_Dataloader = [DataLoader(PDGset, batch_size=16) for PDGset in PDG_Dataset]
MOV_Dataloader = DataLoader(MOV_Dataset, batch_size=16)
# whiteNoisy_Dataloader = DataLoader(whiteNoisy_Dataset, batch_size=16)

In [ ]:
model = FCAE(cellNum*stride)
model.load_state_dict(torch.load('./FCAE/last_08261732_mouse1_day0.ckpt'))
model.to(device)

In [ ]:
PDG_Predictions = [predict(loader, model, device) for loader in PDG_Dataloader]
MOV_Predictions = predict(MOV_Dataloader, model, device)
# whiteNoisy_Predictions = predict(whiteNoisy_Data, model, device)

In [ ]:
PDG_MSE = [getMSE(d, pred.numpy()) for d, pred in zip(PDG_Data, PDG_Predictions)]
MOV_MSE = getMSE(MOV_Data, MOV_Predictions.numpy())
# whiteNoisy_MSE = getMSE(whiteNoisy_Data.numpy(), whiteNoisy_Predictions.numpy())

In [ ]:
PDG_df = [pd.DataFrame(mse, columns=[f'MSE Day {i*7}']) for i, mse in enumerate(PDG_MSE)]
MOV_df = pd.DataFrame(MOV_MSE, columns=[f'MSE'])

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

plt.style.use('bmh')
fig, ax = plt.subplots()
# print(PDG_df)
ax.hist(PDG_df[0][PDG_df[0]['MSE Day 0'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='Day0')
# ax.hist(PDG_df[1][PDG_df[1]['MSE Day 7'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='Day7')
# ax.hist(PDG_df[2][PDG_df[2]['MSE Day 14'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='Day14')
# ax.hist(PDG_df[3][PDG_df[3]['MSE Day 21'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='Day21')
# ax.hist(PDG_df[4][PDG_df[4]['MSE Day 28'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='Day28')
ax.hist(MOV_df[MOV_df['MSE'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='MOV')
# ax.hist(whiteNoise_df[whiteNoise_df['MSE'] < 1.25], histtype="stepfilled", bins=25, alpha=0.8, density=True, label='WN')
ax.set_title("Different Session MSE")

ax.legend(loc='upper left')

plt.show()
# plt.savefig('./LSTMAE_DAY.jpg', dpi=400)